In [1]:
import pulp

############################
# 1) Discrete-Time Scheduler
############################
def schedule_wbs_discrete(tasks, resource_capacities, time_horizon=None):
    """
    Schedules 'tasks' in discrete time steps using integer linear programming.
    
    Args:
        tasks: A list of dicts, each with:
               {
                 "id": <unique string identifier>,
                 "duration": <integer duration in discrete time units>,
                 "dependencies": <list of predecessor task IDs>,
                 "resource": <resource name needed>
               }
        resource_capacities: dict e.g. {"Admin": 3, "LabTech": 1}
        time_horizon: Optional integer upper bound on the schedule length. 
                      If None, we use sum of all durations + len(tasks) as a simple upper bound.
    
    Returns:
        (makespan, start_times)
        - makespan: integer (optimal finishing time for all tasks)
        - start_times: dict { task_id -> integer start time }
    """
    
    # If user doesn't specify a max horizon, pick something safe:
    if time_horizon is None:
        time_horizon = sum(t["duration"] for t in tasks) + len(tasks)
    
    # Gather a simple list of IDs
    task_ids = [t["id"] for t in tasks]
    
    # Build quick lookups
    durations    = {t["id"]: t["duration"]    for t in tasks}
    dependencies = {t["id"]: t["dependencies"] for t in tasks}
    resources    = {t["id"]: t["resource"]     for t in tasks}
    
    # Create the PuLP model
    model = pulp.LpProblem("Discrete_WBS_Scheduling", pulp.LpMinimize)
    
    # Decision variables: x[i, t] = 1 if task i starts at time t
    x = pulp.LpVariable.dicts(
        "start",
        [(i, t) for i in task_ids for t in range(time_horizon + 1)],
        cat=pulp.LpBinary
    )
    
    # Makespan T: integer variable
    T = pulp.LpVariable("Makespan", lowBound=0, upBound=time_horizon, cat=pulp.LpInteger)
    
    # 1) Each task must start exactly once
    for i in task_ids:
        # You can't start a task so late that it won't fit before time_horizon
        max_start = time_horizon - durations[i]
        model += (
            pulp.lpSum(x[(i, t)] for t in range(max_start + 1)) == 1
        ), f"OneStart_{i}"
    
    # 2) Precedence constraints
    #    If task j is a dependency of i, then start(i) >= finish(j).
    for i in task_ids:
        for j in dependencies[i]:
            dur_j = durations[j]
            # For every possible start time t_j of j,
            # i cannot start before t_j + dur_j
            max_start_j = time_horizon - dur_j
            for t_j in range(max_start_j + 1):
                finish_j = t_j + dur_j
                # So if x[j, t_j] = 1, then i must not start < finish_j
                # => for all t_i < finish_j, x[i, t_i] = 0 in combination with x[j,t_j].
                for t_i in range(finish_j):
                    model += x[(j, t_j)] + x[(i, t_i)] <= 1, f"Precedence_{j}_before_{i}_{t_j}_{t_i}"
    
    # 3) Resource capacity constraints
    #    For each resource r, at each time step u, sum of tasks active at u <= capacity.
    #    A task is active at u if it started at t <= u < t + dur[i].
    for r, cap in resource_capacities.items():
        # tasks that need resource r
        tasks_r = [i for i in task_ids if resources[i] == r]
        for u in range(time_horizon + 1):
            active_sum = []
            for i in tasks_r:
                dur_i = durations[i]
                # The earliest start time that can still be active at u is u-dur_i+1
                # We'll just check all t in [0..u] but ensure u < t+dur_i
                for t_val in range(u+1):
                    if t_val + dur_i > u:  # means if started at t_val, task i is still active at time u
                        active_sum.append(x[(i, t_val)])
            model += pulp.lpSum(active_sum) <= cap, f"Resource_{r}_time_{u}"
    
    # 4) Makespan: T >= finish time of each task
    for i in task_ids:
        dur_i = durations[i]
        max_start_i = time_horizon - dur_i
        for t_val in range(max_start_i + 1):
            model += T >= t_val + dur_i * x[(i, t_val)], f"Makespan_{i}_{t_val}"
    
    # Objective: minimize makespan
    model += T, "Minimize_Makespan"
    
    # Solve
    model.solve(pulp.PULP_CBC_CMD(msg=0))
    
    # Extract solution
    start_times = {}
    for i in task_ids:
        st = None
        max_start_i = time_horizon - durations[i]
        for t_val in range(max_start_i + 1):
            if pulp.value(x[(i, t_val)]) > 0.5:
                st = t_val
                break
        start_times[i] = st
    
    makespan_value = int(round(pulp.value(T))) if pulp.value(T) is not None else None
    return makespan_value, start_times

############################
# 2) Utility: Replicate WBS
############################
def replicate_wbs(template, count, prefix):
    """
    Replicates the given WBS template 'count' times. 
    Each copy gets a unique ID: prefix + originalID + _i
    Adjusts dependencies accordingly.
    """
    replicated_tasks = []
    for n in range(count):
        instance_suffix = f"_{n}"
        for task in template:
            old_id = task["id"]
            new_id = prefix + old_id + instance_suffix
            
            # Convert dependencies
            new_deps = []
            for dep_id in task["dependencies"]:
                new_deps.append(prefix + dep_id + instance_suffix)
            
            replicated_tasks.append({
                "id": new_id,
                "duration": task["duration"],
                "dependencies": new_deps,
                "resource": task["resource"]
            })
    return replicated_tasks

####################################
# 3) Compute Resource Usage and Cost
####################################
def compute_resource_usage_cost(tasks, start_times, resource_capacities, resource_unit_costs, makespan):
    """
    Calculates how many tasks are active per resource at each time step,
    and derives a total usage cost.
    
    tasks: list of tasks (with 'id','duration','resource')
    start_times: dict {task_id -> start_time}
    resource_capacities: e.g. {"Admin": 3, "LabTech": 1}
    resource_unit_costs: e.g. {"Admin": 10, "LabTech": 25} cost per "slot" per time step
    makespan: integer solution from the schedule
    """
    durations = {t["id"]: t["duration"] for t in tasks}
    resources = {t["id"]: t["resource"] for t in tasks}
    
    # usage_by_time[r][u] = how many tasks are active using resource r at time u
    usage_by_time = {r: [0]*(makespan+1) for r in resource_capacities.keys()}
    
    # Fill usage
    for t_id, st in start_times.items():
        d = durations[t_id]
        r = resources[t_id]
        # Active in [st, st + d-1]
        for u in range(st, st + d):
            if u <= makespan:
                usage_by_time[r][u] += 1
    
    total_cost = 0.0
    # Summation: cost = sum_{r,u} usage_by_time[r][u] * resource_unit_costs[r]
    for r in usage_by_time:
        for u in range(makespan+1):
            usage_count = usage_by_time[r][u]
            slot_cost = resource_unit_costs.get(r, 0.0)  # default 0 if not in dict
            total_cost += usage_count * slot_cost
    return total_cost, usage_by_time

########################################
# 4) Scenario Building and Running Logic
########################################
def build_scenario_tasks(metals_count, composites_count, polymer_count,
                         metals_template, composites_template, polymer_template):
    # You can replicate each template multiple times
    tasks = []
    tasks += replicate_wbs(metals_template, metals_count,     "MET_")
    tasks += replicate_wbs(composites_template, composites_count, "COMP_")
    tasks += replicate_wbs(polymer_template, polymer_count,   "POLY_")
    return tasks

def run_scenarios(scenario_definitions,
                  metals_template, composites_template, polymer_template,
                  resource_unit_costs):
    """
    scenario_definitions: list of dicts, each with:
        {
            'metals_count': int,
            'composites_count': int,
            'polymer_count': int,
            'resource_capacities': {...}  # e.g. {"Admin":3, "LabTech":2}
        }
    resource_unit_costs: dict, e.g. {"Admin":10, "LabTech":25}
    
    Returns: list of scenario results.
    """
    results = []
    
    for scenario in scenario_definitions:
        # 1) Build all tasks for this scenario
        all_tasks = build_scenario_tasks(
            scenario['metals_count'],
            scenario['composites_count'],
            scenario['polymer_count'],
            metals_template,
            composites_template,
            polymer_template
        )
        
        # 2) Solve discrete scheduling
        makespan, starts = schedule_wbs_discrete(
            tasks=all_tasks,
            resource_capacities=scenario['resource_capacities'],
            time_horizon=None
        )
        
        # 3) Compute usage/cost
        total_cost, usage_by_time = compute_resource_usage_cost(
            tasks=all_tasks,
            start_times=starts,
            resource_capacities=scenario['resource_capacities'],
            resource_unit_costs=resource_unit_costs,
            makespan=makespan
        )
        
        # 4) Aggregate results
        scenario_result = {
            'scenario': scenario,
            'makespan': makespan,
            'start_times': starts,
            'total_resource_cost': total_cost,
            'usage_by_time': usage_by_time
        }
        results.append(scenario_result)
    
    return results

###########################
# EXAMPLE MAIN / DEMO USAGE
###########################
if __name__ == "__main__":
    
    # EXAMPLE TEMPLATES (Adjust as needed)
    metals_template = [
        # Suppose metals analysis has 3 tasks
        {"id": "M1", "duration": 4, "dependencies": [],         "resource": "Admin"},
        {"id": "M2", "duration": 2, "dependencies": ["M1"],     "resource": "LabTech"},
        {"id": "M3", "duration": 3, "dependencies": ["M2"],     "resource": "LabTech"}
    ]
    
    composites_template = [
        # Suppose composites analysis has 2 tasks
        {"id": "C1", "duration": 3, "dependencies": [],         "resource": "Admin"},
        {"id": "C2", "duration": 4, "dependencies": ["C1"],     "resource": "LabTech"}
    ]
    
    polymer_template = [
        # Suppose polymer analysis has 2 tasks
        {"id": "P1", "duration": 2, "dependencies": [],         "resource": "Admin"},
        {"id": "P2", "duration": 5, "dependencies": ["P1"],     "resource": "LabTech"}
    ]
    
    # Example scenarios to run
    scenario_definitions = [
        {
            'metals_count': 2,
            'composites_count': 2,
            'polymer_count': 2,
            'resource_capacities': {"Admin": 2, "LabTech": 1}
        },
        {
            'metals_count': 5,
            'composites_count': 2,
            'polymer_count': 3,
            'resource_capacities': {"Admin": 3, "LabTech": 2}
        },
    ]
    
    # Example resource cost (cost per "slot" per hour)
    resource_unit_costs = {"Admin": 10, "LabTech": 25}
    
    # Run all scenarios
    all_results = run_scenarios(
        scenario_definitions,
        metals_template,
        composites_template,
        polymer_template,
        resource_unit_costs
    )
    
    # Print out results
    for idx, result in enumerate(all_results):
        scn = result['scenario']
        print(f"\n=== Scenario {idx+1} ===")
        print(f"  - Metals: {scn['metals_count']}, Composites: {scn['composites_count']}, Polymers: {scn['polymer_count']}")
        print(f"  - Resource capacities: {scn['resource_capacities']}")
        print(f"  -> Makespan = {result['makespan']}")
        print(f"  -> Total Resource Cost = ${result['total_resource_cost']:.2f}")
        
        # If you want to see individual start times, you can do so:
        # for t_id, st in sorted(result['start_times'].items()):
        #     print(f"    Task {t_id} starts at {st} finishes at {st + next((t['duration'] for t in (metals_template+composites_template+polymer_template) if t['id'] == t_id.split('_')[0]),0)} (approx)")
        #
        # Or break down usage by time:
        # usage = result['usage_by_time']
        # for r, usage_list in usage.items():
        #     print(f"    Resource {r} usage timeline: {usage_list}")



=== Scenario 1 ===
  - Metals: 2, Composites: 2, Polymers: 2
  - Resource capacities: {'Admin': 2, 'LabTech': 1}
  -> Makespan = 58
  -> Total Resource Cost = $880.00

=== Scenario 2 ===
  - Metals: 5, Composites: 2, Polymers: 3
  - Resource capacities: {'Admin': 3, 'LabTech': 2}
  -> Makespan = 103
  -> Total Resource Cost = $1520.00
